In [20]:
from llm.llm.chatgpt import ChatGPT
from setup_db import create_chroma_db, connect_to_db, execute_query, DB
from llm_test import MappingTable, find_suitable_column, reasoning_text2SQL, CoT_reasoning

In [21]:
llm = ChatGPT()

In [22]:
db_name = 'test_db'
user = 'postgres'
password = '12345678'
port = '5433'
host = 'localhost'

In [24]:
db = DB(db_name, user, password, host, port)


Connecting to database test_db, postgres...


In [25]:
collection_chromadb = 'category_bank_chroma'
persist_directory = 'data/category_bank_chroma'
bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_non_bank_chroma'
persist_directory = 'data/category_non_bank_chroma'
none_bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

In [27]:
mapping_table = MappingTable(vector_db_bank=bank_vector_store, vector_db_non_bank=none_bank_vector_store, db=db)

In [28]:
prompt = "Calculate ROA, ROE of the bank VIB in Q1 2024"

In [16]:
find_suitable_column(llm, prompt)

CompletionUsage(completion_tokens=240, prompt_tokens=154, total_tokens=394, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0))
To calculate the Return on Assets (ROA) and Return on Equity (ROE) for a bank such as VIB, we need to focus on specific columns from the financial statement that pertain to the bank's assets and equity.

1. **Return on Assets (ROA)** is calculated using the formula:
   \[
   ROA = \frac{Net \ Income}{Total \ Assets}
   \]
   - For this, we need the "Net Income" and "Total Assets" columns.

2. **Return on Equity (ROE)** is calculated using the formula:
   \[
   ROE = \frac{Net \ Income}{Total \ Equity}
   \]
   - For this, we need the "Net Income" and "Total Equity" columns.

In a bank's financial statement, the relevant columns would typically be labeled as follows:

- "Net Income"
- "Total Assets"
- "Total Equity"

Based on this analysis, the suitable columns for the bank's financial statements are:

```json
{
    "bank_column_name": ["Net 

{'bank_column_name': ['Net Income', 'Total Assets', 'Total Equity'],
 'non_bank_column_name': []}

In [29]:
sql = reasoning_text2SQL(llm, prompt, mapping_table.search_return_df, 5)

CompletionUsage(completion_tokens=233, prompt_tokens=154, total_tokens=387, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0))
To calculate the Return on Assets (ROA) and Return on Equity (ROE) for a bank like VIB in Q1 2024, we need specific data from the bank's financial statements. 

For banks, ROA is typically calculated using the following formula:

- **ROA = Net Income / Total Assets**

ROE is computed using:

- **ROE = Net Income / Total Equity**

The relevant columns in the financial statement for a bank to retrieve the necessary information are:

1. **Net Income**: This is found in the income statement and represents the bank's profit after all expenses have been deducted from revenues.
2. **Total Assets**: This is found in the balance sheet and represents all resources owned by the bank.
3. **Total Equity**: This is also found in the balance sheet and represents the net worth of the bank (Assets - Liabilities).

Based on this analysis, the suggested column

In [30]:
print(sql[0]['code'])

WITH profit_after_tax AS (
    SELECT bfs.data AS net_income, bfs.stock_code, bfs.year, bfs.quarter
    FROM bank_financial_report bfs
    JOIN map_category_code_bank mc
      ON bfs.category_code = mc.category_code
    WHERE bfs.stock_code = 'VIB'
      AND bfs.year = 2024
      AND bfs.quarter = 1
      AND mc.en_caption = 'Profit after Tax'
),
total_assets AS (
    SELECT bfs.data AS total_assets, bfs.stock_code, bfs.year, bfs.quarter
    FROM bank_financial_report bfs
    JOIN map_category_code_bank mc
      ON bfs.category_code = mc.category_code
    WHERE bfs.stock_code = 'VIB'
      AND bfs.year = 2024
      AND bfs.quarter = 1
      AND mc.en_caption = 'Total Assets'
),
equity AS (
    SELECT bfs.data AS equity, bfs.stock_code, bfs.year, bfs.quarter
    FROM bank_financial_report bfs
    JOIN map_category_code_bank mc
      ON bfs.category_code = mc.category_code
    WHERE bfs.stock_code = 'VIB'
      AND bfs.year = 2024
      AND bfs.quarter = 1
      AND mc.en_caption = 'Equi

In [31]:
db.query(sql[0]['code'], return_type='dataframe')

,stock_code,year,quarter,net_income,total_assets,equity,roa,roe
0,VIB,2024,1,2001.143,413887.815,38415.725,0.004835,0.052092


In [12]:
res = bank_vector_store.similarity_search('Asset', 5)
res

[Document(page_content='Other Assets', metadata={'code': 'BS_250', 'lang': 'en'}),
 Document(page_content='Fixed Assets', metadata={'code': 'BS_220', 'lang': 'en'}),
 Document(page_content='Total Assets', metadata={'code': 'BS_300', 'lang': 'en'}),
 Document(page_content='Asset Revaluation Differences', metadata={'code': 'BS_440', 'lang': 'en'}),
 Document(page_content='Equity', metadata={'code': 'BS_500', 'lang': 'en'})]